---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [76]:
import numpy as np
import pandas as pd
import re


def energy():
  
  return energy

def load_energy():
    """
    1. energy = read "Energy Indicators.xls" 
    # note that column C has the country names 
    # with footnote numbers while column B has no footnotes
    2. if you read the C column in 1 then remove any digits in the country column
    3. replace all entries with the value '...' with np.NaN in the energy dataframe
    4. convert 'Energy Supply' column values to giga-jouls
    dicts = {"Republic of Korea": "South Korea",
               "United States of America": "United States",
               "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
               "China, Hong Kong Special Administrative Region": "Hong Kong"}
    5. replace each country in dicts's keys with the corresponding dicts value

    6. remove the part after ' (' form all countries that have paranthesis, *****make sure the result has no trailing whitespace*******

    
    
    energy DF - Energy Indicators.xls
    1. drop first 2 columns & rename column labels w/ ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    2. Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule)
    3. missing data (e.g. data with "...") make sure this is reflected as np.NaN values
    4. Rename countries:
       "Republic of Korea": "South Korea",
       "United States of America": "United States",
       "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
       "China, Hong Kong Special Administrative Region": "Hong Kong"
    5. Remove numbers and/or parenthesis from country names
    """
    # load energy DF
    df = pd.read_excel("./Energy Indicators.xls", skiprows=17)
    
    # after checking the column B, find index w/ content 'Sources:' and remove all rows it anf after
    src_idx = df.index[df['Unnamed: 1'] == 'Sources:'][0]
    df.drop([row_idx for row_idx in range(src_idx - 2, df.shape[0])], inplace=True)

    # drop first 2 column
    df.drop(df.columns[[0, 1]], axis=1, inplace=True)
    
    # replace '...' w/ np.NaN
    df.replace("...", np.NaN, inplace=True)

    # rename column labels
    col_lbl = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    df.columns = col_lbl
    
#     # drop rows for 'Country' is NaN
#     df = df[df['Country'].notnull()]

    # Convert Energy Supply to gigajoules
    df['Energy Supply'] *= 1.0e6
    df['Energy Supply per Capita'] = df['Energy Supply per Capita'].astype(np.float64)
    df['Energy Supply'] = df['Energy Supply'].astype(np.float64)
    df['% Renewable'] = df['% Renewable'].astype(np.float64)

    # remove numbers and/or parenthesis from country names
    countries_lst = list(df['Country'])
    num = re.compile("[0-9]")
    parenth = re.compile("\s(\(.+\))")
    sub_countries = {"Republic of Korea": "South Korea", "United States of America": "United States",
                     "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                     "China, Hong Kong Special Administrative Region": "Hong Kong",
                     'China, Macao Special Administrative Region': 'Macao',
                     'Data exclude Hong Kong and Macao Special Administrative Regions  and Taiwan Province.': 'Taiwan'#,
#                      'Iran ': 'Iran'
                    }
    countries = []
    for country in countries_lst:
        country = num.sub('', country)
        country = parenth.sub('', country)
        cpuntry = country.strip()
        if country in sub_countries.keys():
            country = sub_countries[country]
        countries.append(country)

    df['Country'] = countries
    
    return df


def load_gdp():
    """
    def gdp():

      1.GDP =  read 'world_bank.csv'
      2. replace each country in dicts's keys with the corresponding dicts value
              dicts = {"Korea, Rep.": "South Korea", 
                   "Iran, Islamic Rep.": "Iran",
                   "Hong Kong SAR, China": "Hong Kong"}
      3. change column name "Country Name" to "Country"
      4. keep only the column ['Country', '2006', '2007', '2008','2009','2010', '2011','2012','2013','2014','2015']

      return GDP
    """

    df = pd.read_csv('world_bank.csv', skiprows = 4)
    sub_countries = {"Korea, Rep.": "South Korea",
                     "Iran, Islamic Rep.": "Iran",
                     "Hong Kong SAR, China": "Hong Kong"
                    }
    
    # replace country name
    df['Country Name'] = df['Country Name'].apply(
        lambda country: sub_countries[country] if country in sub_countries.keys() else country)
    
    # keep only the required columns
    cols_lbl = ['Country Name', '2006', '2007', '2008','2009','2010', '2011','2012','2013','2014','2015']
    
    df = df[cols_lbl]
    
    # rename 'Country Name' to 'Country'
    col_names = list(df.columns)
    col_names[0] = 'Country'
    df.columns = col_names

    return df#.set_index(df['Country'])

def load_scimen():
    """
    def scimen()

      1. ScimEn= read 'scimagojr-3.xlsx'

      return ScimEn
    """
    return pd.read_excel('scimagojr-3.xlsx')

def join_tbls(energy, GDP, ScimEn):
    """
    def answer_one():
        #call the function
        energy, GDP, ScimEn = energy(),gdp(),scimen()
        tdf = inner merge GDP and energy on 'Country'
        tdf = inner merge tdf and ScimEn on 'Country'
        res = get the records in tdf where the column Rank value is less or equal 15 (boolean masking)
        return res
    """   
    join_df = pd.merge(ScimEn, pd.merge(energy, GDP, left_on="Country", right_on="Country"), 
                       left_on='Country', right_on='Country'
                      )
             '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

    # Using Country name as index
    join_df = join_df.set_index('Country')

    
    return join_df

def top15():

    energy = load_energy()
    GDP = load_gdp()
    ScimEn = load_scimen()
    
    join_df = join_tbls(energy, GDP, ScimEn)
    
    top15_df = join_df.loc[join_df['Rank'] <= 15]
    
    return top15_df

def answer_one():
    energy = load_energy()
    GDP = load_gdp()
    ScimEn = load_scimen()
    join_df = join_tbls(energy, GDP, ScimEn)
    
    top15_df = join_df.loc[join_df['Rank'] <= 15]
    
#     return top15_df
    return join_df
#     return ScimEn
#     return GDP
#     return energy

# type(answer_one())
answer_one()#.info()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286.0,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124.0,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [1]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [17]:
def union_tbls(energy, GDP, ScumEn):

#     drop_lst = [str(year) for year in range(1960, 2006)]
#     GDP.drop(drop_lst, axis=1, inplace=True)
    
    union_df = pd.merge(ScimEn, pd.merge(energy,  GDP, how='outer', left_on="Country", right_on="Country Name"), 
                       how='outer', left_on='Country', right_on='Country'
                      )

    cols = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 
             'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', 
             '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

    # Using Country name as index
    union_df = union_df.set_index('Country')

    # Select columns w/ given columns labels
    union_df = union_df[cols]
    
#     # select top 15 rnked recorded
#     join_df = join_df.loc[join_df['Rank'] <= 15]
    
    return union_df

def answer_two():
    energy = load_energy()
    GDP = load_gdp()
    ScimEn = load_scimen()

    join_df = join_tbls(energy, GDP, ScimEn)
    join_rows = join_df.shape[0]
#     print(join_rows)

    union_rows = union_tbls(energy, GDP, ScimEn).shape[0]
#     print(union_rows)
    
    return (union_rows - join_rows)

# answer_two()

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [28]:
def answer_three():
    
    Top15 = answer_one()
    
    years = [str(year) for year in range(2006, 2016)]
    
    result = Top15.apply(lambda x: np.mean(x[years]), axis=1)
    
    return result

# answer_three()

Country
China                 6.348609e+12
United States         1.536434e+13
Japan                 5.542208e+12
United Kingdom        2.487907e+12
Russian Federation    1.565459e+12
Canada                1.660647e+12
Germany               3.493025e+12
India                 1.769297e+12
France                2.681725e+12
South Korea           1.106715e+12
Italy                 2.120175e+12
Spain                 1.418078e+12
Iran                  4.441558e+11
Australia             1.164043e+12
Brazil                2.189794e+12
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [51]:
def answer_four():
    Top15 = answer_one()
    return Top15.iloc[6]['2015'] - Top15.iloc[6]['2006']

# answer_four()

352665152317.87988

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [52]:
def answer_five():
    Top15 = answer_one()
    return np.mean(Top15['Energy Supply per Capita'])

# answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [9]:
def answer_six():
    Top15 = answer_one()
    return (Top15[Top15['% Renewable'] == Top15['% Renewable'].max()].index[0], Top15['% Renewable'].max())

# answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [18]:
def answer_seven():
    Top15 = answer_one()
    
    Top15['Citation Ratio'] = Top15['Self-citations'] / Top15['Citations']
    
#     return Top15
    return (Top15[Top15['Citation Ratio'] == Top15['Citation Ratio'].max()].index[0], Top15['Citation Ratio'].max())


# answer_seven()

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [42]:
def answer_eight():
    Top15 = answer_one()
    
    Top15['Est. Popul.'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    
    new = Top15.sort_values(by='Est. Popul.', ascending=False)
    
    return new.iloc[2].name

# answer_eight()

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [50]:
def answer_nine():
    Top15 = answer_one()
    
    return "ANSWER"

# answer_nine()

In [44]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [52]:
# plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    Top15 = answer_one()
    return "ANSWER"

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():
    Top15 = answer_one()
    return "ANSWER"

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    Top15 = answer_one()
    return "ANSWER"

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    Top15 = answer_one()
    return "ANSWER"

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!